## Campbell e Schiller (1991)

Usando dados do Futuro DI

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [2]:
# mostra o dataframe inteiro
# pd.set_option('display.max_rows', None)

Para a base de dados, usaremos o futuro DI que funciona como uma proxy para o retorno das Letra do Tesouro Nacional (LTN). Isso porque o futuro DI é medido através da expectativa da taxa de juros livre de risco da economia, caracaterizada pelo título LTN do governo que representa um ativo livre de risco.

Os dados do futuro DI estão em periodicidade mensal, e abrangem todo o período a partir de Junho de 2012 até Outubro de 2022. Os dados foram obtidos através da Reuters cujo acesso é liberado pelo departamento de Economia da PUC-Rio.

In [3]:
futurodi = pd.read_csv('futuroDI.csv', sep=',', index_col=0)

In [4]:
# função que torna o índice um objeto de data
futurodi.index = pd.to_datetime(futurodi.index)

In [5]:
# normalizando os dados que estavam em taxas para números decimais
futurodi = futurodi/100

In [6]:
# dropando colunas que não serão usadads
futurodi = futurodi.drop(['BRPRE2Y=BMF', 'BRPRE3Y=BMF', 'BRPRE4Y=BMF', 'BRPRE5Y=BMF', 'BRPRE6Y=BMF', 'BRPRE7Y=BMF', 'BRPRE8Y=BMF', 'BRPRE9Y=BMF', 'BRPRE10Y=BMF'], axis = 1)

In [7]:
futurodi

,BRPRE1M=BMF,BRPRE2M=BMF,BRPRE3M=BMF,BRPRE4M=BMF,BRPRE5M=BMF,BRPRE6M=BMF,BRPRE7M=BMF,BRPRE8M=BMF,BRPRE9M=BMF,BRPRE10M=BMF,BRPRE11M=BMF,BRPRE1Y=BMF
2022-10-31,0.13659,0.13670,0.13688,0.136990,0.136990,0.13672,0.136560,0.136050,0.134820,0.133880,0.132860,0.13130
2022-09-30,0.13668,0.13682,0.13689,0.137620,0.136540,0.13688,0.136780,0.136080,0.135580,0.134800,0.133410,0.13245
2022-08-31,0.13673,0.13708,0.13743,0.137450,0.137540,0.13760,0.137490,0.137740,0.137380,0.136730,0.136130,0.13526
2022-07-31,0.13513,0.13619,0.13724,0.137960,0.138360,0.13879,0.138940,0.139150,0.139150,0.139160,0.138770,0.13823
2022-06-30,0.13144,0.13330,0.13479,0.135930,0.137010,0.13782,0.138770,0.139000,0.139230,0.139630,0.139710,0.13923
...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-31,0.07096,0.07099,0.07089,0.070720,0.071000,0.07116,0.071130,0.071230,0.071530,0.071840,0.072010,0.07252
2012-09-30,0.07293,0.07272,0.07257,0.072440,0.072470,0.07259,0.079137,0.080644,0.082187,0.083416,0.084623,0.07466
2012-08-31,0.07348,0.07308,0.07287,0.072650,0.072560,0.07259,0.077958,0.080215,0.081829,0.083180,0.084323,0.07468
2012-07-31,0.07806,0.07648,0.07511,0.074270,0.073770,0.07348,0.078232,0.080065,0.081631,0.082856,0.083541,0.07431


A yield $y_t^{(n)}$ é o rendimento anualizado da LTN comprada em $t$ que paga R\$1 em $t+n$. Como o futuro DI é uma proxy do retorno da LTN. Como $BRPREnM=BMF$ é o futuro DI anualizado entre $t$ e $t+n$, então temos que

$$
y_t^{(n)} = BRPREnM=BMF
$$

In [8]:
futurodi = futurodi.rename(columns={'BRPRE1M=BMF':'y_t(1)',
                                    'BRPRE2M=BMF':'y_t(2)',
                                    'BRPRE3M=BMF':'y_t(3)',
                                    'BRPRE4M=BMF':'y_t(4)',
                                    'BRPRE5M=BMF':'y_t(5)',
                                    'BRPRE6M=BMF':'y_t(6)',
                                    'BRPRE7M=BMF':'y_t(7)',
                                    'BRPRE8M=BMF':'y_t(8)',
                                    'BRPRE9M=BMF':'y_t(9)',
                                    'BRPRE10M=BMF':'y_t(10)',
                                    'BRPRE11M=BMF':'y_t(11)',
                                    'BRPRE1Y=BMF':'y_t(12)'})

Precisamos também definir $y_{t+1}^{(n-1)}$ como o retorno da LTN comprada em $t+1$ que paga R\$1 em $t+1+(n-1) = t+n$

In [9]:
futurodi['y_t+1(2-1)'] = futurodi['y_t(1)'].shift(1)
futurodi['y_t+1(3-1)'] = futurodi['y_t(2)'].shift(1)
futurodi['y_t+1(4-1)'] = futurodi['y_t(3)'].shift(1)
futurodi['y_t+1(6-1)'] = futurodi['y_t(5)'].shift(1)
futurodi['y_t+1(9-1)'] = futurodi['y_t(8)'].shift(1)
futurodi['y_t+1(12-1)'] = futurodi['y_t(11)'].shift(1)

In [10]:
futurodi

,y_t(1),y_t(2),y_t(3),y_t(4),y_t(5),y_t(6),y_t(7),y_t(8),y_t(9),y_t(10),y_t(11),y_t(12),y_t+1(2-1),y_t+1(3-1),y_t+1(4-1),y_t+1(6-1),y_t+1(9-1),y_t+1(12-1)
2022-10-31,0.13659,0.13670,0.13688,0.136990,0.136990,0.13672,0.136560,0.136050,0.134820,0.133880,0.132860,0.13130,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-30,0.13668,0.13682,0.13689,0.137620,0.136540,0.13688,0.136780,0.136080,0.135580,0.134800,0.133410,0.13245,0.13659,0.13670,0.13688,0.13699,0.136050,0.132860
2022-08-31,0.13673,0.13708,0.13743,0.137450,0.137540,0.13760,0.137490,0.137740,0.137380,0.136730,0.136130,0.13526,0.13668,0.13682,0.13689,0.13654,0.136080,0.133410
2022-07-31,0.13513,0.13619,0.13724,0.137960,0.138360,0.13879,0.138940,0.139150,0.139150,0.139160,0.138770,0.13823,0.13673,0.13708,0.13743,0.13754,0.137740,0.136130
2022-06-30,0.13144,0.13330,0.13479,0.135930,0.137010,0.13782,0.138770,0.139000,0.139230,0.139630,0.139710,0.13923,0.13513,0.13619,0.13724,0.13836,0.139150,0.138770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-31,0.07096,0.07099,0.07089,0.070720,0.071000,0.07116,0.071130,0.071230,0.071530,0.071840,0.072010,0.07252,0.07089,0.07057,0.07044,0.07047,0.070620,0.071210
2012-09-30,0.07293,0.07272,0.07257,0.072440,0.072470,0.07259,0.079137,0.080644,0.082187,0.083416,0.084623,0.07466,0.07096,0.07099,0.07089,0.07100,0.071230,0.072010
2012-08-31,0.07348,0.07308,0.07287,0.072650,0.072560,0.07259,0.077958,0.080215,0.081829,0.083180,0.084323,0.07468,0.07293,0.07272,0.07257,0.07247,0.080644,0.084623
2012-07-31,0.07806,0.07648,0.07511,0.074270,0.073770,0.07348,0.078232,0.080065,0.081631,0.082856,0.083541,0.07431,0.07348,0.07308,0.07287,0.07256,0.080215,0.084323


Defina 

$$
s_t^{(n)} = y_t^{(n)} - y_t^{(1)}
$$

In [11]:
futurodi['s_t(2)'] = futurodi['y_t(2)'] - futurodi['y_t(1)']
futurodi['s_t(3)'] = futurodi['y_t(3)'] - futurodi['y_t(1)']
futurodi['s_t(4)'] = futurodi['y_t(4)'] - futurodi['y_t(1)']
futurodi['s_t(6)'] = futurodi['y_t(6)'] - futurodi['y_t(1)']
futurodi['s_t(9)'] = futurodi['y_t(9)'] - futurodi['y_t(1)']
futurodi['s_t(12)'] = futurodi['y_t(12)'] - futurodi['y_t(1)']

O primeiro modelo de Campbell e Schiller (1991) é mostrado a seguir

$$
y_{t+1}^{(n-1)} - y_t^{(n)} = \alpha_n + \beta_n \frac{s_t^{(n)}}{n-1} + \epsilon_t^{(n)}
$$

onde $n \in \{2,3,4,6,9,12\}$

In [12]:
futurodi['y'] = futurodi['y_t+1(2-1)'] - futurodi['y_t(2)']
futurodi['x'] = futurodi['s_t(2)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n = 2$, a regressão é

$$
y_{t+1}^{(1)} - y_t^{(2)} = \alpha_2 + \beta_2 s_t^{(2)} + \epsilon_t^{(2)}
$$

In [13]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     213.2
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.52e-28
Time:                        19:45:03   Log-Likelihood:                 644.40
No. Observations:                 124   AIC:                            -1285.
Df Residuals:                     122   BIC:                            -1279.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003      0.000     -2.419      0.0

In [14]:
futurodi['y'] = futurodi['y_t+1(3-1)'] - futurodi['y_t(3)']
futurodi['x'] = futurodi['s_t(3)']/2
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n = 3$, a regressão é

$$
y_{t+1}^{(2)} - y_t^{(3)} = \alpha_3 + \beta_3 \frac{s_t^{(3)}}{2} + \epsilon_t^{(3)}
$$

In [15]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     109.5
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.11e-18
Time:                        19:45:03   Log-Likelihood:                 603.80
No. Observations:                 124   AIC:                            -1204.
Df Residuals:                     122   BIC:                            -1198.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0005      0.000     -2.832      0.0

In [16]:
futurodi['y'] = futurodi['y_t+1(4-1)'] - futurodi['y_t(4)']
futurodi['x'] = futurodi['s_t(4)']/3
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n = 4$, a regressão é

$$
y_{t+1}^{(3)} - y_t^{(4)} = \alpha_4 + \beta_4 \frac{s_t^{(4)}}{3} + \epsilon_t^{(4)}
$$

In [17]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     62.21
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.48e-12
Time:                        19:45:04   Log-Likelihood:                 575.28
No. Observations:                 124   AIC:                            -1147.
Df Residuals:                     122   BIC:                            -1141.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0004      0.000     -1.954      0.0

In [18]:
futurodi['y'] = futurodi['y_t+1(6-1)'] - futurodi['y_t(6)']
futurodi['x'] = futurodi['s_t(6)']/5
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n = 6$, a regressão é

$$
y_{t+1}^{(5)} - y_t^{(6)} = \alpha_6 + \beta_6 \frac{s_t^{(6)}}{5} + \epsilon_t^{(6)}
$$

In [19]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     41.46
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.48e-09
Time:                        19:45:04   Log-Likelihood:                 535.69
No. Observations:                 124   AIC:                            -1067.
Df Residuals:                     122   BIC:                            -1062.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0005      0.000     -1.629      0.1

In [20]:
futurodi['y'] = futurodi['y_t+1(9-1)'] - futurodi['y_t(9)']
futurodi['x'] = futurodi['s_t(9)']/8
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n = 9$, a regressão é

$$
y_{t+1}^{(8)} - y_t^{(9)} = \alpha_9 + \beta_9 \frac{s_t^{(9)}}{8} + \epsilon_t^{(9)}
$$

In [21]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     19.34
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.35e-05
Time:                        19:45:04   Log-Likelihood:                 493.43
No. Observations:                 124   AIC:                            -982.9
Df Residuals:                     122   BIC:                            -977.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0006      0.000     -1.390      0.1

In [22]:
futurodi['y'] = futurodi['y_t+1(12-1)'] - futurodi['y_t(12)']
futurodi['x'] = futurodi['s_t(12)']/11
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n = 12$, a regressão é

$$
y_{t+1}^{(11)} - y_t^{(12)} = \alpha_{12} + \beta_{12} \frac{s_t^{(12)}}{11} + \epsilon_t^{(12)}
$$

In [23]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     13.26
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           0.000400
Time:                        19:45:04   Log-Likelihood:                 475.50
No. Observations:                 124   AIC:                            -947.0
Df Residuals:                     122   BIC:                            -941.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0002      0.001     -0.370      0.7

O segundo modelo é

$$
\sum_{i=1}^{n} \left(1 - \frac{i}{n}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(n)} - y_t^{(1)}] + v_t^{(n)}
$$

onde $n \in \{2,3,4,6,9,12\}$

Precisamos então de mais algumas yields. Defina $y_{t+i}^{(1)}$ como o retorno da LTN comprada em $t+i$ que paga R\$1 em $t+i+1$

In [24]:
futurodi['y_t+2(1)'] = futurodi['y_t(1)'].shift(2)
futurodi['y_t+3(1)'] = futurodi['y_t(1)'].shift(3)
futurodi['y_t+4(1)'] = futurodi['y_t(1)'].shift(4)
futurodi['y_t+5(1)'] = futurodi['y_t(1)'].shift(5)
futurodi['y_t+6(1)'] = futurodi['y_t(1)'].shift(6)
futurodi['y_t+7(1)'] = futurodi['y_t(1)'].shift(7)
futurodi['y_t+8(1)'] = futurodi['y_t(1)'].shift(8)
futurodi['y_t+9(1)'] = futurodi['y_t(1)'].shift(9)
futurodi['y_t+10(1)'] = futurodi['y_t(1)'].shift(10)
futurodi['y_t+11(1)'] = futurodi['y_t(1)'].shift(11)

In [25]:
futurodi['y'] = (1/2)*(futurodi['y_t+1(2-1)'] - futurodi['y_t(1)'])
futurodi['x'] = futurodi['s_t(2)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n=2$, a regressão é

$$
\sum_{i=1}^{2} \left(1 - \frac{i}{2}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(2)} - y_t^{(1)}] + v_t^{(2)}
$$

$$
\frac{1}{2}\left(y_{t+1}^{(1)} - y_{t}^{(1)}\right) = \alpha + \beta[y_t^{(2)} - y_t^{(1)}] + v_t^{(2)}
$$

In [26]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     867.1
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.79e-57
Time:                        19:45:05   Log-Likelihood:                 730.35
No. Observations:                 124   AIC:                            -1457.
Df Residuals:                     122   BIC:                            -1451.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0001   6.19e-05     -2.419      0.0

In [27]:
futurodi['y'] = (2/3)*(futurodi['y_t+1(2-1)'] - futurodi['y_t(1)']) + (1/3)*(futurodi['y_t+2(1)']-futurodi['y_t+1(2-1)'])
futurodi['x'] = futurodi['s_t(3)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n=3$, a regressão é

$$
\sum_{i=1}^{3} \left(1 - \frac{i}{3}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(3)} - y_t^{(1)}] + v_t^{(3)}
$$

$$
\frac{2}{3}\left(y_{t+1}^{(1)} - y_{t}^{(1)}\right) + \frac{1}{3}\left(y_{t+2}^{(1)} - y_{t+1}^{(1)}\right) = \alpha + \beta[y_t^{(3)} - y_t^{(1)}] + v_t^{(3)}
$$

In [28]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     818.9
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.06e-55
Time:                        19:45:05   Log-Likelihood:                 640.73
No. Observations:                 123   AIC:                            -1277.
Df Residuals:                     121   BIC:                            -1272.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0004      0.000     -3.474      0.0

In [29]:
futurodi['y'] = (3/4)*(futurodi['y_t+1(2-1)'] - futurodi['y_t(1)']) + (2/4)*(futurodi['y_t+2(1)']-futurodi['y_t+1(2-1)']) + (1/4)*(futurodi['y_t+3(1)']-futurodi['y_t+2(1)'])
futurodi['x'] = futurodi['s_t(4)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n=4$, a regressão é

$$
\sum_{i=1}^{4} \left(1 - \frac{i}{4}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(4)} - y_t^{(1)}] + v_t^{(4)}
$$

$$
\frac{3}{4}\left(y_{t+1}^{(1)} - y_{t}^{(1)}\right) + \frac{2}{4}\left(y_{t+2}^{(1)} - y_{t+1}^{(1)}\right) + \frac{1}{4}\left(y_{t+3}^{(1)} - y_{t+2}^{(1)}\right) = \alpha + \beta[y_t^{(4)} - y_t^{(1)}] + v_t^{(4)}
$$

In [30]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                     761.9
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           8.31e-54
Time:                        19:45:06   Log-Likelihood:                 584.63
No. Observations:                 122   AIC:                            -1165.
Df Residuals:                     120   BIC:                            -1160.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0007      0.000     -3.446      0.0

In [31]:
futurodi['y'] = (5/6)*(futurodi['y_t+1(2-1)'] - futurodi['y_t(1)']) + (4/6)*(futurodi['y_t+2(1)']-futurodi['y_t+1(2-1)']) + (3/6)*(futurodi['y_t+3(1)']-futurodi['y_t+2(1)']) + (2/6)*(futurodi['y_t+4(1)']-futurodi['y_t+3(1)']) + (1/6)*(futurodi['y_t+5(1)']-futurodi['y_t+4(1)'])
futurodi['x'] = futurodi['s_t(6)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n=6$, a regressão é

$$
\sum_{i=1}^{6} \left(1 - \frac{i}{6}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(6)} - y_t^{(1)}] + v_t^{(6)}
$$

$$
\frac{5}{6}\left(y_{t+1}^{(1)} - y_{t}^{(1)}\right) + \frac{4}{6}\left(y_{t+2}^{(1)} - y_{t+1}^{(1)}\right) + \frac{3}{6}\left(y_{t+3}^{(1)} - y_{t+2}^{(1)}\right) + \frac{2}{6}\left(y_{t+4}^{(1)} - y_{t+3}^{(1)}\right) + \frac{1}{6}\left(y_{t+5}^{(1)} - y_{t+4}^{(1)}\right) = \alpha + \beta[y_t^{(6)} - y_t^{(1)}] + v_t^{(6)}
$$

In [32]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     611.5
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.67e-48
Time:                        19:45:06   Log-Likelihood:                 505.48
No. Observations:                 120   AIC:                            -1007.
Df Residuals:                     118   BIC:                            -1001.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0012      0.000     -3.482      0.0

In [33]:
futurodi['y'] = (8/9)*(futurodi['y_t+1(2-1)'] - futurodi['y_t(1)']) + (7/9)*(futurodi['y_t+2(1)']-futurodi['y_t+1(2-1)']) + (6/9)*(futurodi['y_t+3(1)']-futurodi['y_t+2(1)']) + (5/9)*(futurodi['y_t+4(1)']-futurodi['y_t+3(1)']) + (4/9)*(futurodi['y_t+5(1)']-futurodi['y_t+4(1)']) + (3/9)*(futurodi['y_t+6(1)']-futurodi['y_t+5(1)']) + (2/9)*(futurodi['y_t+7(1)']-futurodi['y_t+6(1)']) + (1/9)*(futurodi['y_t+8(1)']-futurodi['y_t+7(1)'])
futurodi['x'] = futurodi['s_t(9)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n=9$, a regressão é

$$
\sum_{i=1}^{9} \left(1 - \frac{i}{9}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(9)} - y_t^{(1)}] + v_t^{(9)}
$$

In [34]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     430.7
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.09e-40
Time:                        19:45:07   Log-Likelihood:                 425.70
No. Observations:                 117   AIC:                            -847.4
Df Residuals:                     115   BIC:                            -841.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0025      0.001     -3.929      0.0

In [35]:
futurodi['y'] = (11/12)*(futurodi['y_t+1(2-1)'] - futurodi['y_t(1)']) + (10/12)*(futurodi['y_t+2(1)']-futurodi['y_t+1(2-1)']) + (9/12)*(futurodi['y_t+3(1)']-futurodi['y_t+2(1)']) + (8/12)*(futurodi['y_t+4(1)']-futurodi['y_t+3(1)']) + (7/12)*(futurodi['y_t+5(1)']-futurodi['y_t+4(1)']) + (6/12)*(futurodi['y_t+6(1)']-futurodi['y_t+5(1)']) + (5/12)*(futurodi['y_t+7(1)']-futurodi['y_t+6(1)']) + (4/12)*(futurodi['y_t+8(1)']-futurodi['y_t+7(1)']) + (3/12)*(futurodi['y_t+9(1)']-futurodi['y_t+8(1)']) + (2/12)*(futurodi['y_t+10(1)']-futurodi['y_t+9(1)']) + (1/12)*(futurodi['y_t+11(1)']-futurodi['y_t+10(1)'])
futurodi['x'] = futurodi['s_t(12)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $n=12$, a regressão é

$$
\sum_{i=1}^{12} \left(1 - \frac{i}{12}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(12)} - y_t^{(1)}] + v_t^{(12)}
$$

In [36]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     372.6
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.03e-37
Time:                        19:45:07   Log-Likelihood:                 378.34
No. Observations:                 114   AIC:                            -752.7
Df Residuals:                     112   BIC:                            -747.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0034      0.001     -3.897      0.0